# Tutorial 1

more detailed docs: https://marctoussaint.github.io/robotics-course/

In [1]:
import sys
sys.path.append('../../../build')
import numpy as np
import libry as ry

**ry-c++-log** /home/jay/git/optimization-course/rai/rai/ry/ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback



## Setting up a basic Config

The starting point is to create a `Configuration`.

In [2]:
C = ry.Config()
C.view()

This shows an empty configuration. Tip: Make the view window appear "Always On Top" (right click on the window bar)

You can add things (objects, scene models, robots) to a configuration.

In [3]:
C.clear()
C.addFile('../../../rai-robotModels/pr2/pr2.g')
C.addFile('../../../rai-robotModels/objects/kitchen.g')

Note that the view was updated automatically.

In [19]:
ball = C.addFrame(name="ball")
ball.setShape(ry.ST.sphere, [.1])
ball.setPosition([.8,.8,1.5])
ball.setColor([1,1,0])

One can also add convex meshes (just passing the vertex array), or use sphere-swept convex meshes (ssBox, capsule, sphere, etc)

In [4]:
hand = C.addFrame(name="hand", parent="pr2L")
hand.setShape(ry.ST.ssBox, size=[.2,.2,.1,.02]),
hand.setRelativePosition([0,0,-.1])
hand.setColor([1,1,0])

In this last example, the new object has another frame (pr2L) as *parent*. This means that it is permanently attached to this parent. pos and quat/rot are interpreted relative to the parent.

In [6]:
f = C.frame("hand")
print("position:", f.getPosition())
print("orientation:", f.getQuaternion())

position: [0.15365539 0.56413119 0.96711955]
orientation: [ 0.33464431 -0.27540941  0.72687736 -0.53274023]


## Joint and Frame State

A configuration is a tree of n frames. Every frame has a pose (position & quaternion), which is represented as a 7D vector (x,y,z, qw,qx,qy,qz). The frame state is the $n\times 7$ matrix, where the i-th row is the pose of the i-th frame.

A configuration also defines joints, which means that the relative transfromation from a parent to a child frame is parameterized by degrees-of-freedoms (DOFs). If the configuration has in total n DOFs, the joint state is a n-dimensional vector.

Setting the joint state implies computing all relative transformations, and then forward chaining all transformations to compute all frame poses. So setting the joint state also sets the frame state.
     
Setting the frame state allows you to set frame poses that are inconsistent/impossible w.r.t. the joints! Setting the frame state implies computing all relative transformations from the frame poses, and then assigning the joint state to the *projection* onto the actual DOFs

In [7]:
q = C.getJointState()
print('joint names: ', C.getJointNames() )
print('joint state: ', q)

joint names:  ['worldTranslationRotation:0', 'worldTranslationRotation:1', 'worldTranslationRotation:2', 'torso_lift_joint', 'head_pan_joint', 'laser_tilt_mount_joint', 'r_shoulder_pan_joint', 'l_shoulder_pan_joint', 'head_tilt_joint', 'r_shoulder_lift_joint', 'l_shoulder_lift_joint', 'r_upper_arm_roll_joint', 'l_upper_arm_roll_joint', 'r_elbow_flex_joint', 'l_elbow_flex_joint', 'r_forearm_roll_joint', 'l_forearm_roll_joint', 'r_wrist_flex_joint', 'l_wrist_flex_joint', 'r_wrist_roll_joint', 'l_wrist_roll_joint', 'r_gripper_l_finger_jointr_gripper_r_finger_jointr_gripper_l_finger_tip_jointr_gripper_r_finger_tip_joint', 'l_gripper_l_finger_jointl_gripper_r_finger_jointl_gripper_l_finger_tip_jointl_gripper_r_finger_tip_joint', 'r_gripper_joint', 'l_gripper_joint']
joint state:  [ 0.          0.          1.57079633  0.1         0.          0.
 -1.          1.          0.4         0.5         0.5        -1.
  1.         -2.         -2.         -1.5         1.5        -0.5
 -0.5        -0.5 

Let's move the configuration by adding to the joint configuration

In [9]:
q[2] = q[2] + 1.
C.setJointState(q)

The *frame state* is a $n\times 7$ matrix, which contains for all of $n$ frames the 7D pose. A pose is stored as [p_x, p_y, p_z, q_w, q_x, q_y, q_z], with position p and quaternion q.

In [31]:
X0 = C.getFrameState()
print('frame state: ', X0)

frame state:  [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  7.07106781e-01]
 [ 1.11022302e-17 -5.00000000e-02  8.90675000e-01 ...  5.00000000e-01
  -5.00000000e-01  5.00000000e-01]
 ...
 [ 5.00000000e-01  2.00000000e+00  4.00000000e-01 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 8.00000000e-01  8.00000000e-01  1.50000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-1.53655388e-01  5.64131186e-01  9.67119554e-01 ... -2.75409412e-01
  -7.26877359e-01  5.32740227e-01]]


Let's do a questionable thing: adding .1 to all numbers in the frame matrix!

In [36]:
X = X0 + .1
C.setFrameState(X)

The rows of X have non-normalized quaternions! These are normalized when setting the frame state.

Also, the frame poses are now *inconsistent* to the joint constraints! We can read out the projected joint state, set the joint state, and get a consistent state again:

In [37]:
C.setJointState( C.getJointState() )

Now all *joint* transformations are consistent: just hingeX transformations or alike. However, all the other relative transformations of links and shapes are still messed up from setting their frame pose. Let's bring the configuration back into the state before the harsh *setFrame*

In [38]:
C.setFrameState(X0)

 ## Selecting joints

Often one would like to choose which joints are actually active, that is, which joints are referred to in q. This allows one to sub-select joints and work only with projections of the full configuration state. This changes the joint state dimensionality, including ordering of entries in q.

The frame state is not affected by such a selection of active joints.

In [39]:
C.selectJointsByTag(["armL","base"])
q = C.getJointState()
print('joint state: ', q)
print('joint names: ', C.getJointNames() )

joint state:  [ 0.          0.          1.57079633  1.00001     0.50001     1.00001
 -2.00001     1.50001    -0.50001     0.50001   ]
joint names:  ['worldTranslationRotation:0', 'worldTranslationRotation:1', 'worldTranslationRotation:2', 'l_shoulder_pan_joint', 'l_shoulder_lift_joint', 'l_upper_arm_roll_joint', 'l_elbow_flex_joint', 'l_forearm_roll_joint', 'l_wrist_flex_joint', 'l_wrist_roll_joint']


## Features & Jacobians

A core part of rai defines features over configurations. A feature is a differentiable mapping from a configuration (or set of configurations) to a vector. Starndard features are "position-of-endeffector-X" or "distance/penetration-between-convex-shapes-A-and-B", etc. But there are many, many more features defined in rai, like error of Newton-Euler-equations for an object, total energy of the system, etc. Defining differentiable features is the core of many functionalities in the rai code.

Let's define a basic feature over C: the 3D (world coordinate) position of pr2L (left hand)

In [14]:
F = C.feature(ry.FS.position, ["pr2L"])

We can now evaluate the feature, and also get the Jacobian:

In [15]:
print(F.description(C))

[y,J] = F.eval(C)
print('hand position:', y)
print('Jacobian:', J)
print('Jacobian shape:', J.shape)

F_Position/0-pr2L
hand position: [-0.2316488   0.50511666  0.94627935]
Jacobian: [[ 1.00000000e+00  0.00000000e+00 -5.05116662e-01  2.22044605e-16
   0.00000000e+00  0.00000000e+00  0.00000000e+00 -5.55116662e-01
   0.00000000e+00  0.00000000e+00 -4.67894440e-02  0.00000000e+00
   2.66598983e-01  0.00000000e+00 -2.88521368e-01  0.00000000e+00
  -5.44613844e-02  0.00000000e+00 -7.28240781e-02  0.00000000e+00
  -1.38777878e-17  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00 -2.31648796e-01  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00 -4.36487962e-02
   0.00000000e+00  0.00000000e+00  3.00431565e-02  0.00000000e+00
   2.16456798e-02  0.00000000e+00  4.04226887e-01  0.00000000e+00
   8.25709454e-02  0.00000000e+00  2.51342878e-02  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
   0.00000000e+00  0.00

We can linearly transform features by setting 'scale' and 'target':

In [16]:
F.setScale([10.]) #note: needs to be an array!
F.setTarget([0., 0., 1.])
F.eval(C)

(array([-2.31648796,  5.05116662, -0.53720654]),
 array([[ 1.00000000e+01,  0.00000000e+00, -5.05116662e+00,
          2.22044605e-15,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00, -5.55116662e+00,  0.00000000e+00,
          0.00000000e+00, -4.67894440e-01,  0.00000000e+00,
          2.66598983e+00,  0.00000000e+00, -2.88521368e+00,
          0.00000000e+00, -5.44613844e-01,  0.00000000e+00,
         -7.28240781e-01,  0.00000000e+00, -1.38777878e-16,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  1.00000000e+01, -2.31648796e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00, -4.36487962e-01,  0.00000000e+00,
          0.00000000e+00,  3.00431565e-01,  0.00000000e+00,
          2.16456798e-01,  0.00000000e+00,  4.04226887e+00,
          0.00000000e+00,  8.25709454e-01,  0.00000000e+00,
          2.51342878e-01,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00

Setting scale and target actually transforms the feature to become
$$
  \phi(x) \gets \texttt{scale} \cdot (\phi(x) - \texttt{target})
$$
The $\cdot$ is flexibly interpreted: scale can be an arbitrary matrix, or a vector of same size as $\phi$ (element-wise re-scaling), or a single scalar (as above). E.g., if we only care about the height of the robot hand, we can do this:

In [17]:
F.setScale([[0,0,10]])      #pick z-coordinate only, and scale by 10
F.setTarget([0., 0., 1.])   #note that the target needs to be in UNSCALED space!
F.eval(C)

(array([-0.53720654]),
 array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+01,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  1.23260660e-15,  0.00000000e+00,
          0.00000000e+00, -2.36660008e+00,  0.00000000e+00,
         -3.89235088e+00,  0.00000000e+00, -1.17134468e+00,
          0.00000000e+00, -3.00023881e-01,  0.00000000e+00,
          2.01366038e+00,  0.00000000e+00,  4.16333634e-16,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00]]))

A negative-distance (penetration) feature example

In [20]:
F2 = C.feature(ry.FS.distance, ["hand", "ball"]) #distance actually means neg-distance
print(F2.description(C))

F_PairCollision/0-hand-ball


In [21]:
F2.eval(C)

(array([-0.63534625]),
 array([[ 0.75395208,  0.28526308, -0.3749512 ,  0.59176114, -0.        ,
         -0.        , -0.46627826, -0.        , -0.        , -0.03685205,
         -0.        ,  0.50471875, -0.        ,  0.31674125, -0.        ,
         -0.04690754, -0.        ,  0.27225257, -0.        ,  0.02244167,
         -0.        , -0.        , -0.        , -0.        , -0.        ]]))

When a feature is of higher *order*, by default it computes the difference, acceleration, etc, w.r.t. multiple configurations. We need to create a compound configurations ("tuple") of several configurations to evaluate this:

In [22]:
C2 = ry.Config()
C2.addConfigurationCopy(C)  #this replicates the whole structure
C2.addConfigurationCopy(C)  #this replicates the whole structure
V2 = C2.view()

In [23]:
F.setScale([])  #reset to default (no scaling)
F.setTarget([]) #reset to default (zero target)
F.setOrder(1)
F.eval(C2)[0]

array([0., 0., 0.])

This should be zero, as the difference between the first and second copy is zero. To see a difference, let's move the 2nd configuration:

In [24]:
q = C.getJointState()
q = q - .1
C2.setJointStateSlice(q,1)
y = F.eval(C2)[0]
print('hand difference (y(C2) - y(K)) =', y)

hand difference (y(C2) - y(K)) = [ 0.01638086 -0.14782551 -0.05227247]


An acceleration example:

In [25]:
C2.addConfigurationCopy(C);

In [26]:
C2.setJointStateSlice(q + .3, 2);

In [27]:
F.setOrder(2)
(y,J) = F.eval(C2)
print('hand acceleration:', y)
print('shape of Jacobian:', J.shape)
print(F.description(C2))

hand acceleration: [-0.08686053  0.46826691  0.1609393 ]
shape of Jacobian: (3, 75)
F_Position/2-pr2L


Note that the Jacobian is now w.r.t. all three configurations! It is of size 3x(3xdim(q)). Let's retrieve the Jacobian w.r.t. C3 only:

In [28]:
J = J.reshape((3,3,q.size))
print('shape of Jacobian:', J.shape)
J[:,1,:]

shape of Jacobian: (3, 3, 25)


array([[-2.00000000e+00,  0.00000000e+00,  9.14582306e-01,
        -1.11022302e-16,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  9.76545358e-01,  0.00000000e+00,
         0.00000000e+00,  1.48251505e-01,  0.00000000e+00,
        -4.58640670e-01,  0.00000000e+00,  3.54588252e-01,
         0.00000000e+00,  6.47946543e-02,  0.00000000e+00,
         9.23445085e-02,  0.00000000e+00, -3.46944695e-18,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00, -2.00000000e+00,  2.30535870e-01,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -1.53569038e-01,  0.00000000e+00,
         0.00000000e+00, -1.43984026e-01,  0.00000000e+00,
        -2.08757539e-02,  0.00000000e+00, -9.33495798e-01,
         0.00000000e+00, -2.31056474e-01,  0.00000000e+00,
        -4.86315948e-02,  0.00000000e+00,  5.55111512e-17,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
    

In [29]:
C=0

## Editing Configurations

In [30]:
C = ry.Config()
C.view()

In [31]:
import subprocess
subprocess.Popen(['gedit', 'tmp.g'])

In [ ]:
C.edit('tmp.g')

**ry-c++-log** /home/jay/git/optimization-course/rai/rai/Core/util.cpp:getIs:1081(-1) could not open file 'tmp.g' for input from '/home/jay/git/optimization-course/rai/test/ry./.'

**ry-c++-log** /home/jay/git/optimization-course/rai/rai/Core/util.cpp:getIs:1081(-1) could not open file 'tmp.g' for input from '/home/jay/git/optimization-course/rai/test/ry./.'

**ry-c++-log** /home/jay/git/optimization-course/rai/rai/Core/util.cpp:getIs:1081(-1) could not open file 'tmp.g' for input from '/home/jay/git/optimization-course/rai/test/ry./.'

**ry-c++-log** /home/jay/git/optimization-course/rai/rai/Core/util.cpp:getIs:1081(-1) could not open file 'tmp.g' for input from '/home/jay/git/optimization-course/rai/test/ry./.'

**ry-c++-log** /home/jay/git/optimization-course/rai/rai/Core/util.cpp:getIs:1081(-1) could not open file 'tmp.g' for input from '/home/jay/git/optimization-course/rai/test/ry./.'

**ry-c++-log** /home/jay/git/optimization-course/rai/rai/Core/util.cpp:getIs:1081(-1) could not


**ry-c++-log** /home/jay/git/optimization-course/rai/rai/Core/util.cpp:getIs:1081(-1) could not open file 'tmp.g' for input from '/home/jay/git/optimization-course/rai/test/ry./.'

**ry-c++-log** /home/jay/git/optimization-course/rai/rai/Core/util.cpp:getIs:1081(-1) could not open file 'tmp.g' for input from '/home/jay/git/optimization-course/rai/test/ry./.'

**ry-c++-log** /home/jay/git/optimization-course/rai/rai/Core/util.cpp:getIs:1081(-1) could not open file 'tmp.g' for input from '/home/jay/git/optimization-course/rai/test/ry./.'

**ry-c++-log** /home/jay/git/optimization-course/rai/rai/Core/util.cpp:getIs:1081(-1) could not open file 'tmp.g' for input from '/home/jay/git/optimization-course/rai/test/ry./.'

**ry-c++-log** /home/jay/git/optimization-course/rai/rai/Core/util.cpp:getIs:1081(-1) could not open file 'tmp.g' for input from '/home/jay/git/optimization-course/rai/test/ry./.'

**ry-c++-log** /home/jay/git/optimization-course/rai/rai/Core/util.cpp:getIs:1081(-1) could no


**ry-c++-log** /home/jay/git/optimization-course/rai/rai/Core/util.cpp:getIs:1081(-1) could not open file 'tmp.g' for input from '/home/jay/git/optimization-course/rai/test/ry./.'

**ry-c++-log** /home/jay/git/optimization-course/rai/rai/Core/util.cpp:getIs:1081(-1) could not open file 'tmp.g' for input from '/home/jay/git/optimization-course/rai/test/ry./.'

**ry-c++-log** /home/jay/git/optimization-course/rai/rai/Core/util.cpp:getIs:1081(-1) could not open file 'tmp.g' for input from '/home/jay/git/optimization-course/rai/test/ry./.'

**ry-c++-log** /home/jay/git/optimization-course/rai/rai/Core/util.cpp:getIs:1081(-1) could not open file 'tmp.g' for input from '/home/jay/git/optimization-course/rai/test/ry./.'

**ry-c++-log** /home/jay/git/optimization-course/rai/rai/Core/util.cpp:getIs:1081(-1) could not open file 'tmp.g' for input from '/home/jay/git/optimization-course/rai/test/ry./.'

**ry-c++-log** /home/jay/git/optimization-course/rai/rai/Core/util.cpp:getIs:1081(-1) could no

In [ ]:
C=0